In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import datetime

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [ ]:
inicio = datetime.datetime.now()

In [ ]:
propiedades = pd.read_csv('../../set_datos_one_hot_encoder.csv')
set_pruebas = pd.read_csv('../set_pruebas_one_hot_encoder.csv')

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', \
            'pileta', 'cochera','transporte','comercio', 'servicios', 'doble piso', 'jardin']

In [ ]:
def encodear(cadena):
    "Convierte un string de lista de numeros en una lista de numeros"
    return [float(x) for x in cadena.strip('[').strip(']').split(',')]

def obtener_lista_datos(x):
    return list(x[:3]) + x[3] + list(x[4:])

In [ ]:
propiedades.loc[:,'property_type_encoded'] = propiedades.loc[:,'property_type_encoded']\
.apply(lambda x: encodear(x))

set_pruebas.loc[:,'property_type_encoded'] = set_pruebas.loc[:,'property_type_encoded']\
.apply(lambda x: encodear(x))

# Parametros del estimador

In [ ]:
criterion = 'mse'
max_depth = None
max_features = 0.9

# Con un set de validacion

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(propiedades.loc[:,columnas], \
                                        propiedades.loc[:,'price_aprox_usd'], test_size=0.2)
X_train = pd.DataFrame(X_train).reset_index(drop = True)
X_test = pd.DataFrame(X_test).reset_index(drop = True)

In [ ]:
datos = []

for i in range(len(X_train)): 
    x = []
    for n in columnas:
        x.append(X_train[n][i])
    datos.append(obtener_lista_datos(x))
    
test = []

for i in range(len(X_test)): 
    x = []
    for n in columnas:
        x.append(X_test[n][i])
    test.append(obtener_lista_datos(x))

In [ ]:
tree = DecisionTreeRegressor(criterion = criterion, max_depth = max_depth, max_features = max_features)
    
tree.fit(datos,y_train)
precision = tree.score(test,y_test) * 100
error = mean_squared_error(tree.predict(test), y_test)
print("Precision = {:.2f} %,  Error = {:.0f}".format(precision,error))

# Con el set de pruebas

In [ ]:
set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

In [ ]:
datos = []

for i in range(len(set_entrenamiento_datos)): 
    x = []
    for n in columnas:
        x.append(set_entrenamiento_datos[n][i])
    datos.append(obtener_lista_datos(x))

In [ ]:
arbol = DecisionTreeRegressor(criterion = criterion, max_depth = max_depth, max_features = max_features)

arbol.fit(datos, set_entrenamiento_resultado)

set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,columnas].apply\
                            (lambda x: arbol.predict(obtener_lista_datos(x))[0],axis = 1)

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/AlgoritmoFinal.csv', index = False)

In [ ]:
fin = datetime.datetime.now()
tiempo = fin - inicio
print("Tardo {:.0f} minutos.".format(tiempo.seconds / 60))